In [1]:
import pandas as pd
import yfinance as yf
import statsmodels.api as sm
import getFamaFrenchFactors as gff

In [2]:
ticker = 'AAPL'
start = '2016-1-1'
end = '2022-1-1'

In [3]:
df = yf.download(ticker, start, end, adjusted=True)

[*********************100%***********************]  1 of 1 completed


In [4]:
stock_prices = df['Adj Close']
stock_prices = stock_prices.resample("1M").last()
returns = stock_prices.pct_change()
returns = returns.dropna()
returns.name = "Asset"

In [5]:
ff3_monthly = pd.DataFrame(gff.famaFrench3Factor(frequency='m'))
ff3_monthly.rename(columns={"date_ff_factors": 'Date'}, inplace=True)
ff3_monthly.set_index('Date', inplace=True)
ff_data = ff3_monthly.merge(returns, on='Date')
rf = ff_data['RF'].mean()
market_premium = ff_data['Mkt-RF'].mean()

In [6]:
X = ff_data['Mkt-RF']
ff_data['Asset-RF'] = ff_data['Asset'] - ff_data['RF']
y = ff_data['Asset-RF']
c = sm.add_constant(X)
capm_model = sm.OLS(y, c)
result = capm_model.fit()
print(result.summary())
intercept, beta = result.params

                            OLS Regression Results                            
Dep. Variable:               Asset-RF   R-squared:                       0.365
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     39.13
Date:                Thu, 06 Jan 2022   Prob (F-statistic):           3.03e-08
Time:                        22:49:12   Log-Likelihood:                 90.840
No. Observations:                  70   AIC:                            -177.7
Df Residuals:                      68   BIC:                            -173.2
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0149      0.008      1.770      0.0

In [7]:
expected_return = rf + beta*market_premium
print("Expected monthly returns: " + str(expected_return))
yearly_return = expected_return * 12
print("Expected yearly returns: " + str(yearly_return))

Expected monthly returns: 0.017416957404766495
Expected yearly returns: 0.20900348885719794
